# Using the Pytorch Operator For Katib Hyperparameter Tuning

Using PyTorch to build a model with two convolutional layers and two fully connected layers to perform the multi-class classification of images provided.


### Requirement
All you need is this notebook running in Kubeflow Notebook Server once you have cloned this repo
You can use the pre-packaged Docker image here but if  you choose to build your own docker image, you must also have a Docker client installed on your machine.

## Prerequisites
Before we proceed, check to see if Pytorch is already installed in your Notebook Server Environment or not

In [ ]:
# confirm if torch is installed
! pip list | grep torch

In [ ]:
# if torch is not installed, run this
! pip3 install --user ipywidgets 
! pip3 install --user torch torchvision matplotlib --no-cache-dir

In [1]:
TRAINER_FILE = 'iris.py'

## How to Train the Pytorch Model in the Notebook

Since we ultimately want to train the model in a distributed mode, we put all the code in a single cell.
That way we can save the file and include it in a container image:

In [4]:
%%writefile $TRAINER_FILE
from __future__ import print_function

import numpy as np
import pandas as pd
import argparse
import logging
import os

from torchvision import datasets, transforms
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

WORLD_SIZE = int(os.environ.get("WORLD_SIZE", 1))


iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(4, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return F.log_softmax(x, dim=1)


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            msg = "Train Epoch: {} [{}/{} ({:.0f}%)]\tloss={:.4f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())
            logging.info(msg)
            niter = epoch * len(train_loader) + batch_idx


def test(args, model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.max(1, keepdim=True)[1]  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    logging.info("{{metricName: accuracy, metricValue: {:.4f}}};{{metricName: loss, metricValue: {:.4f}}}\n".format(
        float(correct) / len(test_loader.dataset), test_loss))


def should_distribute():
    return dist.is_available() and WORLD_SIZE > 1


def is_distributed():
    return dist.is_available() and dist.is_initialized()


def main():
    # Training settings
    parser = argparse.ArgumentParser(description="PyTorch MNIST Example")
    parser.add_argument("--batch-size", type=int, default=64, metavar="N",
                        help="input batch size for training (default: 64)")
    parser.add_argument("--test-batch-size", type=int, default=1000, metavar="N",
                        help="input batch size for testing (default: 1000)")
    parser.add_argument("--epochs", type=int, default=30, metavar="N",
                        help="number of epochs to train (default: 10)")
    parser.add_argument("--lr", type=float, default=0.01, metavar="LR",
                        help="learning rate (default: 0.01)")
    parser.add_argument("--momentum", type=float, default=0.5, metavar="M",
                        help="SGD momentum (default: 0.5)")
    parser.add_argument("--no-cuda", action="store_true", default=False,
                        help="disables CUDA training")
    parser.add_argument("--seed", type=int, default=1, metavar="S",
                        help="random seed (default: 1)")
    parser.add_argument("--log-interval", type=int, default=10, metavar="N",
                        help="how many batches to wait before logging training status")
    parser.add_argument("--log-path", type=str, default="",
                        help="Path to save logs. Print to StdOut if log-path is not set")
    parser.add_argument("--save-model", action="store_true", default=False,
                        help="For Saving the current Model")

    if dist.is_available():
        parser.add_argument("--backend", type=str, help="Distributed backend",
                            choices=[dist.Backend.GLOO, dist.Backend.NCCL, dist.Backend.MPI],
                            default=dist.Backend.GLOO)
    args = parser.parse_args()

    # Use this format (%Y-%m-%dT%H:%M:%SZ) to record timestamp of the metrics.
    # If log_path is empty print log to StdOut, otherwise print log to the file.
    if args.log_path == "":
        logging.basicConfig(
            format="%(asctime)s %(levelname)-8s %(message)s",
            datefmt="%Y-%m-%dT%H:%M:%SZ",
            level=logging.DEBUG)
    else:
        logging.basicConfig(
            format="%(asctime)s %(levelname)-8s %(message)s",
            datefmt="%Y-%m-%dT%H:%M:%SZ",
            level=logging.DEBUG,
            filename=args.log_path)

    use_cuda = not args.no_cuda and torch.cuda.is_available()
    if use_cuda:
        print("Using CUDA")

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    if should_distribute():
        print("Using distributed PyTorch with {} backend".format(args.backend))
        dist.init_process_group(backend=args.backend)

    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    # train dataset
    train_dataset = torch.utils.data.TensorDataset(Tensor(X_train),Tensor(y_train).type(torch.LongTensor)) 
    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                              batch_size=args.batch_size, 
                                              shuffle=True, **kwargs)

    # test dataset
    test_dataset = torch.utils.data.TensorDataset(Tensor(X_test),Tensor(y_test).type(torch.LongTensor)) 
    test_loader = torch.utils.data.DataLoader(test_dataset, 
                                              batch_size=args.test_batch_size, 
                                              shuffle=False, **kwargs)

    model = Net().to(device)

    if is_distributed():
        Distributor = nn.parallel.DistributedDataParallel if use_cuda \
            else nn.parallel.DistributedDataParallelCPU
        model = Distributor(model)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader, epoch)

    if (args.save_model):
        torch.save(model.state_dict(), "mnist_cnn.pt")


if __name__ == "__main__":
    main()

Overwriting iris.py


In [6]:
%run $TRAINER_FILE --epochs 20 --log-interval 128

2021-03-25T02:30:53Z INFO     Train Epoch: 1 [0/120 (0%)]	loss=1.0757
2021-03-25T02:30:54Z INFO     {metricName: accuracy, metricValue: 0.2667};{metricName: loss, metricValue: 1.0962}

2021-03-25T02:30:54Z INFO     Train Epoch: 2 [0/120 (0%)]	loss=1.0640
2021-03-25T02:30:54Z INFO     {metricName: accuracy, metricValue: 0.2667};{metricName: loss, metricValue: 1.0812}

2021-03-25T02:30:54Z INFO     Train Epoch: 3 [0/120 (0%)]	loss=1.0526
2021-03-25T02:30:54Z INFO     {metricName: accuracy, metricValue: 0.3333};{metricName: loss, metricValue: 1.0658}

2021-03-25T02:30:54Z INFO     Train Epoch: 4 [0/120 (0%)]	loss=1.0664
2021-03-25T02:30:54Z INFO     {metricName: accuracy, metricValue: 0.5667};{metricName: loss, metricValue: 1.0501}

2021-03-25T02:30:54Z INFO     Train Epoch: 5 [0/120 (0%)]	loss=1.0209
2021-03-25T02:30:54Z INFO     {metricName: accuracy, metricValue: 0.7000};{metricName: loss, metricValue: 1.0348}

2021-03-25T02:30:54Z INFO     Train Epoch: 6 [0/120 (0%)]	loss=1.0092
2021-